## Master in Business Analytics &amp; Big Data - Master's thesis
### "Sales prediction and anomaly detection in times of COVID-19 using data from the stock market and the international press”

Full PDF can be read at this link:
<br>
https://www.linkedin.com/posts/sergio-carrasco-sanchez_final-masters-project-activity-6746471092820475904-YLCE

## Data loading and cleaning
### Sales data load

In [4]:
!pip install --upgrade pip

     |████████████████████████████████| 1.7 MB 24.0 MB/s eta 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 21.2.4
    Uninstalling pip-21.2.4:
      Successfully uninstalled pip-21.2.4


In [5]:
!pip install pandas

import pandas as pd

We proceed to load sales data from CSV file.

In [2]:
sales_data_df = pd.read_csv('./datasets/orders.csv')

In [3]:
sales_data_df

,id,date,user,payment_method,total,subtotal,discount,shipping_costs
0,1,2014-05-02 18:32:01,10614,bank_transfer,7.85,2.95,0.0,4.9
1,2,2014-05-02 19:01:55,10614,bank_transfer,8.19,3.29,0.0,4.9
2,3,2014-05-05 15:39:38,10615,cash_on_delivery,10.65,5.75,0.0,4.9
3,7,2014-05-12 14:48:47,10619,bank_transfer,7.39,2.49,0.0,4.9
4,8,2014-05-12 14:52:46,10619,cash_on_delivery,8.00,3.10,0.0,4.9
...,...,...,...,...,...,...,...,...
119375,128481,2020-10-31 21:16:04,68077,cash_on_delivery,13.25,8.35,0.0,4.9
119376,128482,2020-10-31 21:30:12,68078,cash_on_delivery,16.80,11.90,0.0,4.9
119377,128483,2020-10-31 22:00:58,41554,credit_card,52.86,47.96,0.0,4.9
119378,128484,2020-10-31 22:13:14,64540,paypal,23.25,18.35,0.0,4.9
